In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from Vocab import *
from model import AttentionModel

import pandas as pd
import os
import string
import random

In [14]:
df = pd.read_csv('stsds.csv')

In [15]:
df = df.sample(frac=1)
df.head()

,sentence_A,sentence_B,relatedness_score
3128,One bearded man is standing by a shopping cart...,A man is holding a sign and is seeking money,0.70
1020,A cat is playing with a watermelon,One cat is playing with a watermelon,0.98
678,Two men are leaving the stage,The men are performing on stage,0.70
4290,The orange rider is driving a motorcycle on on...,A man on a yellow sport bike is doing a wheeli...,0.52
2217,Nobody is holding a hedgehog,Someone is holding a hedgehog,0.84


In [16]:
#Hyperparams
lr = 0.2
gamma = 0.95
embed_size = 128
hidden_size = 256
max_vocab_size = 5000
num_epochs = 20
random_sentence_prob = 0.02
dne_prob = 0.0
max_len = 50

In [17]:
textcat = open('stsds-cat.txt').read()
textcat[:400]

'a group of kids is playing in a yard and an old man is standing in the background a group of boys in a yard is playing and a man is standing in the background a group of children is playing in the house and there is no man standing in the background a group of kids is playing in a yard and an old man is standing in the background the young boys are playing outdoors and the man is smiling nearby th'

In [18]:
vocab = Vocabulary(textcat)

In [19]:
vocab_size = vocab.size()
print(vocab_size)

2394


In [20]:
model = AttentionModel(embed_size, hidden_size, max_vocab_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=lr)

In [21]:
for e in range(num_epochs):
    total_loss = 0
    for i in range(len(df)):
        optimizer.zero_grad()
        
        t_a = torch.tensor(vocab.getSentenceArray(df["sentence_A"][i], dne_prob))
        t_b = torch.tensor(vocab.getSentenceArray(df["sentence_B"][i], dne_prob))
        ans = torch.tensor(df["relatedness_score"][i])
        
        out = model(t_a, t_b)
        loss = criterion(out, ans.unsqueeze(0).unsqueeze(0))
        loss.backward()
        total_loss += loss.item()
        
        optimizer.step()

    for param_group in optimizer.param_groups:
        param_group['lr'] *= gamma

    print("Epoch", e, "Loss", total_loss)

/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  corr_attn_params = F.softmax(self.lin_attn(corr).view(1, -1))
/home/krypt/myStuff/pytorch/AnswerRatingPredictor/model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  test_attn_params = F.softmax(self.lin_attn(test).view(1, -1))
/home/krypt/.local/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0 Loss 108.15131793554414
Epoch 1 Loss 67.85420791013149
Epoch 2 Loss 53.49150598473163
Epoch 3 Loss 46.025025424470385


KeyboardInterrupt: 

In [ ]:
torch.save(model, "saved_models/stsds8.pt")

In [ ]:
mini = 1
for i in range(len(df)):
    if df["relatedness_score"][i] < mini:
        mini = df["relatedness_score"][i]
print(mini)

In [ ]:
vocab.vocab[:100]